# Decorator
Adding behavior without altering the class itself.

* Motivation
  - Want to augment an object with additional functionality
  - Do not want to rewrite or alter existing code (OCP)
  - Want to keep new functionality separate (SRP)
  - Need to be able to interact with existing structures
  - Two Options
    - Inherit from required object (if possible)
    - Build a Decorator, which simply references the decorated object.

* Facilitates the addition of behaviors to individual objects without inheriting from them.

* Summary
  - A decorator keeps the reference to the decorated objects
  - Adds utility attributes and methods to augment the object’s features
  - Proxying of underlying calls can be done dynamically
  - Python’s functional decorators wrap functions; no direct relation to the Decorator pattern.
    - i.e., the python decorator overwrites the functionality


## Python Functional Decorator != Decorator Pattern

In [2]:
import time

def some_op():
  print('Starting op')
  time.sleep(1)
  print('We are done')
  return 123

some_op()

Starting op
We are done


123

In [3]:
# To measture the execution time of a func, User Wrapper

import time

def time_it(func):
  def wrapper():
    start = time.time()
    result = func()
    end = time.time()
    print(f'{func.__name__} took {int((end-start)*1000)}ms')
    return result
  return wrapper

# decorator = time_it(some_op)  # make a decorator
# decorator()  # calling it
time_it(some_op)()


Starting op
We are done
some_op took 1000ms


123

In [7]:
# Use Built-in Decorator

@time_it
def half_op(a = 1, b = 2):
  print('Starting op %s', a)
  time.sleep(1)
  print('We are done %s', b)
  return 123

half_op()

Starting op %s 1
We are done %s 2
half_op took 1008ms


123

## Classic Decorator

In [20]:
from abc import ABC


class Shape(ABC):
    def __str__(self):
        return ''


class Circle(Shape):
    def __init__(self, radius=0.0):
        self.radius = radius

    def resize(self, factor):
        self.radius *= factor

    def __str__(self):
        return f'A circle of radius {self.radius}'


class Square(Shape):
    def __init__(self, side):
        self.side = side

    def __str__(self):
        return f'A square with side {self.side}'


class ColoredShape(Shape):
    def __init__(self, shape, color):
        # To prevent double applications
        if isinstance(shape, ColoredShape):
            raise Exception('Cannot apply ColoredDecorator twice')
        self.shape = shape
        self.color = color

    def __str__(self):
        return f'{self.shape} has the color {self.color}'


class TransparentShape(Shape):
    def __init__(self, shape, transparency):
        self.shape = shape
        self.transparency = transparency

    def __str__(self):
        return f'{self.shape} has {self.transparency * 100.0}% transparency'




In [23]:

circle = Circle(2)
print(circle)

red_circle = ColoredShape(circle, "red")
print(red_circle)

# ColoredShape doesn't have resize()
# red_circle.resize(3)

red_half_transparent_square = TransparentShape(red_circle, 0.5)
print(red_half_transparent_square)

# nothing prevents double application
# twice = ColoredShape(ColoredShape(Circle(3), 'red'), 'blue')
# print(twice)

# T(C(T(shape)) -> Hard to catch
twice = ColoredShape(TransparentShape(ColoredShape(TransparentShape(Circle(3), 0.5), 'blue'), 0.3), 'red')
print(twice)



A circle of radius 2
A circle of radius 2 has the color red
A circle of radius 2 has the color red has 50.0% transparency
A circle of radius 3 has 50.0% transparency has the color blue has 30.0% transparency has the color red


## Dynamic Decorator

In [44]:
class FileWithLogging:
  def __init__(self, file):
    self.file = file

  def writelines(self, strings):
    self.file.writelines(strings)
    print(f'wrote {len(strings)} lines')  # Add Logging (Decorator)

#   def __iter__(self):
#     return self.file.__iter__()

#   def __next__(self):
#     return self.file.__next__()

  def __getattr__(self, item):  # Get all attrictue of a normal file
    return getattr(self.__dict__['file'], item)

#   def __setattr__(self, key, value):
#     if key == 'file':
#       self.__dict__[key] = value
#     else:
#       setattr(self.__dict__['file'], key)

#   def __delattr__(self, item):
#     delattr(self.__dict__['file'], item)

In [43]:
file = FileWithLogging(open('hello.txt', 'w'))
file.writelines(['hello\n', 'world\n'])
file.write('testing')
file.close()


wrote 2 lines


## Decorator Coding Exercise
You are given two types, Circle and Square, and a decorator called ColoredShape.

The decorator adds the color to the string output for a given shape, just as we did in the lecture.

There's a trick though: the decorator now has a resize() method that should resize the underlying shape. However, only the Circle has a resize() method; the Square does not — do not add it!

You are asked to complete the implementation of Circle, Square and ColoredShape.

In [49]:
class Circle:
  def __init__(self, radius):
    self.radius = radius

  def resize(self, factor):
    self.radius *= factor

  def __str__(self):
    return 'A circle of radius %s' % self.radius

class Square:
  def __init__(self, side):
    self.side = side

  def __str__(self):
    return 'A square with side %s' % self.side
  
class ColoredShape:
  def __init__(self, shape, color):
    self.color = color
    self.shape = shape

  def resize(self, factor):
    # TODO ---------------------------------------------
    r = getattr(self.shape, 'resize', None)
    if callable(r):  # True in Circle, False in Square
      self.shape.resize(factor)

  def __str__(self):
    return "%s has the color %s" %\
           (self.shape, self.color)

In [50]:
from unittest import TestCase
import unittest

class Evaluate(TestCase):
  def test_circle(self):
    circle = ColoredShape(Circle(5), 'red')
    self.assertEqual(
      'A circle of radius 5 has the color red',
      str(circle)
    )
    circle.resize(2)
    self.assertEqual(
      'A circle of radius 10 has the color red',
      str(circle)
    )

  def test_no_resize_in_square(self):
    square = Square(4)
    r = getattr(square, 'resize', None)
    self.assertFalse(callable(r),
                     'Please do not add resize() to Square')

  def test_square(self):
    square = ColoredShape(Square(2), 'blue')
    self.assertEqual(
      'A square with side 2 has the color blue',
      str(square)
    )
    square.resize(2)
    self.assertEqual(
      'A square with side 2 has the color blue',
      str(square)
    )

unittest.main(argv=[''], verbosity=2, exit=False)

test_circle (__main__.Evaluate) ... ok
test_no_resize_in_square (__main__.Evaluate) ... ok
test_square (__main__.Evaluate) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK
